> ### Importando packages

In [1]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install pandera

In [2]:
import pandas as pd
import pandera as pa # A Statistical Data Validation Toolkit for Pandas

> ### Lendo order_payments_dataset.cvs

In [3]:
df_pagamentos = pd.read_csv("../data/order_payments_dataset.csv")
df_pagamentos.rename(columns={'order_id':'id_pedido', 'payment_sequential':'sequencia_pagamento', 'payment_type':'tipo_pagamento', 'payment_installments':'parcelas', 'payment_value':'valor_pagamento'}, inplace=True) # renomeando para português

> ### Buscando inconsistências order_payments_dataset.cvs

In [4]:
df_pagamentos.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id_pedido            103886 non-null  object 
 1   sequencia_pagamento  103886 non-null  int64  
 2   tipo_pagamento       103886 non-null  object 
 3   parcelas             103886 non-null  int64  
 4   valor_pagamento      103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [5]:
df_pagamentos.loc[df_pagamentos.duplicated() == True, ['id_pedido']]

,id_pedido


In [6]:
df_pagamentos.tipo_pagamento = df_pagamentos.tipo_pagamento.str.lower() # setando caixa baixa para cidades

In [7]:
df_pagamentos.isna().any() # verificando a nulidade dos dados

id_pedido              False
sequencia_pagamento    False
tipo_pagamento         False
parcelas               False
valor_pagamento        False
dtype: bool

> ### Trabalhando a tipagem dos dados order_payments_dataset.cvs

In [8]:
df_pagamentos.dtypes

id_pedido               object
sequencia_pagamento      int64
tipo_pagamento          object
parcelas                 int64
valor_pagamento        float64
dtype: object

In [9]:
df_pagamentos.tipo_pagamento.unique()

array(['credit_card', 'boleto', 'voucher', 'debit_card', 'not_defined'],
      dtype=object)

In [10]:
df_pagamentos.replace(['not_defined'], pd.NA, inplace=True)
df_pagamentos.dropna(inplace=True)

In [11]:
# criando schema de validação com pandera
schema_pagamentos = pa.DataFrameSchema(
    columns={
        "id_pedido": pa.Column(pa.String, nullable=False, required=True),
        "sequencia_pagamento": pa.Column(pa.Int, pa.Check.greater_than_or_equal_to(0), nullable=False, required=True),
        "tipo_pagamento": pa.Column(pa.String, pa.Check.isin(['credit_card', 'boleto', 'voucher', 'debit_card']), nullable=False, required=True),
        "parcelas": pa.Column(pa.Int, pa.Check.greater_than_or_equal_to(0), nullable=False, required=True),
        "valor_pagamento": pa.Column(pa.Float, pa.Check.greater_than_or_equal_to(0), nullable=False, required=True),        
    },
)

In [12]:
schema_pagamentos.validate(df_pagamentos) # validando schema_vendedores

,id_pedido,sequencia_pagamento,tipo_pagamento,parcelas,valor_pagamento
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


> ### Exportando para clean_order_payments_dataset.cvs

In [13]:
df_pagamentos.to_csv(r'../data/clean_order_payments_dataset.csv', index = False, header=True)